# Programming for Data Science

## Numpy: Numeric Python

<img src="Resource/ndarray.png" width="500">

- Fundamental Object: ndarray

- Vectorized operations on arrays

- Broadcasting

- File IO and memory-mapped files

- 1D Array
  - 1d_array = np.array([1,2,3])

- 2D Array
  - 2d_array = np.array([[1,2,3], [4,5,6]])
  
- 3D Array
  - 3d_array = np.array([[[1,2,3], [4,5,6]], [[11,12,13], [14,15,16]]])

## Pandas

<img src="Resource/series-and-dataframe.png" width="500">

 - Series: 1D labeled array capable of holding any data type with axis labels or index

- DataFrame: two-dimensional labeled data structures with columns of potentially different types, including:
  - DataFrame
  - Series
  - Numpy ndarray
  - Dictionaries of ndarrays, lists, dictionaries or Series

## Matplotlib

- 2D plotting library which produces publication quality figures

  - Line plots, scatter plots, histograms, pie charts, etc.


- Useable in Python scripts, the Python and IPython shells, the Jupyter Notebook, etc.

  - Not built upon Numpy/Scipy

## Load libraries and datasets

In [ ]:
# Import all the libraries needed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# Read the csv file using pandas
data = pd.read_csv('nyc-rolling-sales.csv')

# Peak into the dataset
data.head()

## Get rid of useless data by inspection

In [ ]:
# Drop 'UnNamed: 0' column because it seems just to be number of iterations
# Drop 'EASE-MENT' column because it is empty
data = data.drop(['Unnamed: 0', 'EASE-MENT', 'SALE DATE'], axis = 1)
data

In [ ]:
# Check if there is any duplicated entry
data.duplicated(data.columns).value_counts()

In [ ]:
# Drop duplicated entries and keep the last occurence of each
data = data.drop_duplicates(data.columns, keep = 'last')
data

## Further data inspection

In [ ]:
# Take a look at all the columns/features in the dataset
data.info()

In [ ]:
# Conver the columns to its proper data type
data['BUILDING CLASS CATEGORY'] = data['BUILDING CLASS CATEGORY'].astype('category')
data['TAX CLASS AT TIME OF SALE'] = data['TAX CLASS AT TIME OF SALE'].astype('category')
data['TAX CLASS AT PRESENT'] = data['TAX CLASS AT PRESENT'].astype('category')
data['LAND SQUARE FEET'] = pd.to_numeric(data['LAND SQUARE FEET'], errors='coerce')
data['GROSS SQUARE FEET']= pd.to_numeric(data['GROSS SQUARE FEET'], errors='coerce')
data['SALE PRICE'] = pd.to_numeric(data['SALE PRICE'], errors='coerce')
data['BOROUGH'] = data['BOROUGH'].astype('category')

### Replace missing data

In [ ]:
# Check which columns contain null data
data.columns[data.isnull().any()]

In [ ]:
# See the proportion of null data in the columns
col_with_missing_data = data.isnull().sum() / len(data)
col_with_missing_data = col_with_missing_data[col_with_missing_data > 0]
col_with_missing_data

In [ ]:
# Replace null data with the median of the column
data['LAND SQUARE FEET'] = data['LAND SQUARE FEET'].fillna(data['LAND SQUARE FEET'].median())
data['GROSS SQUARE FEET'] = data['GROSS SQUARE FEET'].fillna(data['GROSS SQUARE FEET'].median())

### Take a look at our sale prices

In [ ]:
# # Remove outlier sale prices from our data
# price_q1 = data['SALE PRICE'].quantile(0.25)
# price_q3 = data['SALE PRICE'].quantile(0.75)
# price_IQR = price_q3 - price_q1
# upper = price_q3 + 1.5 * price_IQR
# lower = price_q1 - 1.5 * price_IQR
# data = data[(data['SALE PRICE'] < upper) & (data['SALE PRICE'] > lower)]

# Plot a simple histogram of the sale prices
fig, ax = plt.subplots(figsize = (15, 6))
ax.hist(data['SALE PRICE'], bins = 40)
ax.set_xlabel('Price')
ax.set_ylabel('Number of Sales')
ax.set_title('Sale Price Distribution')

### Clean the data further

In [ ]:
# Check if there is any outlier or useless data in the TOTAL UNIT column
data[["TOTAL UNITS", "SALE PRICE"]].groupby(['TOTAL UNITS'], as_index=False).count().sort_values(by='SALE PRICE', ascending=False)

In [ ]:
# Remove the entry with 0 TOTAL UNIT and the entry with outlier 2261 TOTAL UNIT
data = data[(data['TOTAL UNITS'] > 0) & (data['TOTAL UNITS'] != 2261)] 

In [ ]:
# Get rid of more columns that might not be a factor of sale price
data = data.drop(['ADDRESS', 'NEIGHBORHOOD', 'BUILDING CLASS AT PRESENT', 'BUILDING CLASS AT TIME OF SALE', 'APARTMENT NUMBER'], axis = 1)

## Preprocess the data for training

### Standardize numeric data

In [ ]:
# Select columns of numeric data
numeric_data = data.select_dtypes(include = [np.number])
numeric_data.describe()

In [ ]:
scaler = StandardScaler()
scaler.fit(data[numeric_data.columns])
scaled = scaler.transform(data[numeric_data.columns])
for i, col in enumerate(numeric_data.columns):
       data[col] = scaled[:, i]
data.head()

### One-hot encode categorical data

In [ ]:
categorical_data = data.select_dtypes(exclude = [np.number])
categorical_data.describe()

In [ ]:
# Convert categorical variables into dummy/indicator variables (i.e. one-hot encoding).
one_hot_encoded = pd.get_dummies(data[categorical_data.columns])
one_hot_encoded.info(verbose = True, memory_usage = True, null_counts = True)

In [ ]:
# Remove the original categorical columns and replace them with their corresponding one-hot encoded columns
df = data.drop(categorical_data.columns, axis = 1)
df = pd.concat([df, one_hot_encoded], axis = 1)

In [ ]:
# Let's take a look at what we have so far
df.info()

## Train model with our data to predict our sale prices

In [ ]:
# Split the data into features for training and labels
y_df = df['SALE PRICE']
x_df = df.drop('SALE PRICE', axis = 1)

# What do the shapes tell us?
x_df.shape , y_df.shape

In [ ]:
# Split the data into training and testing set
X_train ,X_test, Y_train , Y_test = train_test_split(x_df , y_df , test_size = 0.3 , random_state =34)

In [ ]:
# Fit our training data to a linear regression model and predict
linreg = LinearRegression()
linreg.fit(X_train, Y_train)
Y_pred = linreg.predict(X_test)

In [ ]:
# Let's see how well our model performs
mean_squared_error(Y_test, Y_pred, multioutput='raw_values')